# Tratamento da base dados

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import io

In [ ]:
base_credit = pd.read_csv('credit_card_clients.csv', header = 1)
base_credit

In [ ]:
figura = plt.figure(figsize=(30,30))
sns.heatmap(base_credit.corr(), annot=True);

In [ ]:
base_credit.columns

In [ ]:
X = base_credit.iloc[:, 1:len(base_credit.columns)].values
X

In [ ]:
X.shape

In [ ]:
X[0]

In [ ]:
# Convertendo para o format aceito no Sagemaker
X = np.array(X).astype('float32')
X[0]

# Configurações do SageMaker

In [ ]:
import sagemaker
import boto3
from sagemaker import Session

In [ ]:
session = sagemaker.Session()
bucket = 'sagemaker-curso-bucket'
subpasta_modelo = 'modelos/credit-card/pca'
subpasta_dataset = 'datasets/credit-card'
key_train = 'credit-card-train-data-pca'
role = "arn:aws:iam::339712791663:role/service-role/AmazonSageMaker-ExecutionRole-20240702T173175"
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, subpasta_dataset, key_train)
output_location = 's3://{}/{}/output'.format(bucket, subpasta_modelo)
print('Role: ', role)
print('Localização da base de dados de treinamento: ', s3_train_data)
print('Modelo final será salvo em: ', output_location)

In [ ]:
import sagemaker.amazon.common as smac
buffer = io.BytesIO()
smac.write_numpy_to_dense_tensor(buffer, X)
buffer.seek(0)

In [ ]:
boto3.resource('s3').Bucket(bucket).Object(os.path.join(subpasta_dataset, 'train', key_train)).upload_fileobj(buffer)

# Treinamento do PCA

In [ ]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/ecr-sa-east-1.html
container = sagemaker.image_uris.retrieve(framework = 'pca', region = boto3.Session().region_name)

In [ ]:
# https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html
pca = sagemaker.estimator.Estimator(image_uri = container,
                                    role = role,
                                    instance_count = 1,
                                    instance_type = 'ml.c4.xlarge',
                                    output_path = output_location,
                                    sagemaker_session = session)

In [ ]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/PCA-reference.html
pca.set_hyperparameters(feature_dim = 30000,
                        num_components = 2, # numero de componentes principais que devem ser calculados
                        mini_batch_size = 200) # numero de linhas enviadas para o treinamento

In [ ]:
s3_train_data

In [ ]:
pca.fit({'train': s3_train_data})

# Redução de dimensionalidade

In [ ]:
pca_predictor = pca.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

In [ ]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
pca_predictor.serializer = CSVSerializer()
pca_predictor.deserializer = JSONDeserializer()

In [ ]:
X[0]

In [ ]:
X0_pca = pca_predictor.predict(X[0])
print(X0_pca)

In [ ]:
X_pca = pca_predictor.predict(X)

In [ ]:
X_pca

In [ ]:
X_pca = np.array([r['projection'] for r in X_pca['projections']])

In [ ]:
X_pca, X_pca.shape

In [ ]:
X_pca = np.array(X_pca).astype('float32')

# Treinamento do k-means

In [ ]:
subpasta_modelo_kmeans = 'modelos/credit-card/kmeans'
output_location_kmeans = 's3://{}/{}'.format(bucket, subpasta_modelo_kmeans)
print('Modelo kmeans será salvo em: ', output_location_kmeans)

In [ ]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/k-means.html
# https://docs.aws.amazon.com/sagemaker/latest/dg/k-means-api-config.html
from sagemaker import KMeans

In [ ]:
kmeans = KMeans(role = role,
                instance_count = 1,
                instance_type = 'ml.c4.xlarge',
                output_path = output_location_kmeans,
                spot_intances = True,
                k = 4)

In [ ]:
kmeans.fit(kmeans.record_set(X_pca))

# Agrupamento com o k-means

In [ ]:
kmeans_predictor = kmeans.deploy(initial_instance_count = 1, instance_type = 'ml.c4.xlarge')

In [ ]:
resultados = kmeans_predictor.predict(X_pca)

In [ ]:
resultados

In [ ]:
rotulos = [r.label['closest_cluster'].float32_tensor.values[0] for r in resultados]

In [ ]:
print(rotulos[0:10])

In [ ]:
np.unique(rotulos, return_counts = True)

In [ ]:
base_credit.shape, len(rotulos)

In [ ]:
base_credit['cluster'] = rotulos

In [ ]:
base_credit

In [ ]:
plt.figure(figsize=(10,10))
sns.scatterplot(x = 'LIMIT_BAL', y = 'BILL_AMT5', data = base_credit, hue = 'cluster', palette = ['red', 'green', 'blue', 'orange']);

In [ ]:
base_credit_3 = base_credit[base_credit['cluster'] == 3.0]
base_credit_3

In [ ]:
base_credit_3.to_csv('base_credit_3.csv')

In [ ]:
# https://aws.amazon.com/pt/mxnet/
# https://aws.amazon.com/pt/mxnet/get-started/
import mxnet as mx

In [ ]:
os.system('tar -zxvf model.tar.gz')

In [ ]:
centroides = mx.ndarray.load('model_algo-1')
centroides

In [ ]:
pca_predictor.delete_endpoint()
kmeans_predictor.delete_endpoint()